In [1]:
from sklearn.metrics import mutual_info_score
#from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, pearsonr
import seaborn as sns
import pandas as pd

# Mathematical Underpinnings - Lab ~~4~~ 5

Tests to verify hipoteses of independence (from Lab 4):

In [2]:
def indep_test_asymptotic(X, Y, stat):

    if stat == "mi":

        stat_value = 2*len(X)*mutual_info_score(X, Y)

    if stat == "chi2":

        test_res = (chi2_contingency(pd.crosstab(X, Y)))
        stat_value = test_res.statistic

    df = (len(np.unique(X)) - 1)*(len(np.unique(Y)) - 1)

    p_value = 1 - chi2.cdf(stat_value, df=df)

    return stat_value, p_value

def indep_test_permutation(X, Y, B, stat="mi"):

    stat_value = mutual_info_score(X, Y)

    condition_p_value = 0
    for b in range(B):
        X_b = np.random.permutation(X)

        stat_value_b = mutual_info_score(X_b, Y)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

## Task 1

In [3]:
# a function which computes CMI

In [67]:
CMI = lambda X, Y, Z: np.sum([mutual_info_score(X[Z == z], Y[Z == z]) * np.mean(Z == z) for z in np.unique(Z)])

# example
X = np.array([1, 2, 3, 4])
Y = np.array([1, 2, 3, 3])
Z = np.array([1, 1, 2, 2])

CMI(X, Y, Z)

0.34657359027997264

### a)

In [ ]:
# CI test based on CMI and asymptotics

In [69]:
def asymptotic_conditional_independence_test(X, Y, Z):
    # stat = "cmi"
    stat_value_2nCMI = 2 * len(X) * CMI(X, Y, Z)
    deg_of_freedom = (len(np.unique(X))-1) * (len(np.unique(Y))-1) * len(np.unique(Z))
    p_value = 1 - chi2.cdf(stat_value_2nCMI, df=deg_of_freedom)
    return stat_value_2nCMI, p_value

### b)

In [70]:
# CI test based on CMI and permutations

In [75]:
def conditional_permutations_of_X(X, Y, Z, B):
    # B - number of permutations
    cmi_original = CMI(X, Y, Z)
    stat_value_2nCMI = 2 * len(X) * cmi_original
    cmi_original_smaller_than_cmi_permuted_counter = 0
    for _ in range(B):
        # for each unique z in Z, permute X separately
        X_b = X.copy() 
        for z in np.unique(Z):
            X_b[Z == z] = np.random.permutation(X[Z == z])

        cmi_permuted = CMI(X_b, Y, Z)
        cmi_original_smaller_than_cmi_permuted_counter += cmi_original <= cmi_permuted
            
    p_value = (1 + cmi_original_smaller_than_cmi_permuted_counter) / (1 + B)
    return stat_value_2nCMI, p_value

### c)

In [80]:
N = 1000

conditionaly independent

In [120]:
X = np.round(np.random.normal(0, 1, N))
Y = np.round(np.random.normal(0, 1, N))
Z = np.random.choice([1,2,3,4], N)

test_perm_results = conditional_permutations_of_X(X, Y, Z, B=100)
test_asymptotic_results = asymptotic_conditional_independence_test(X, Y, Z)

print(f"Conditional Mutual Information: {CMI(X,Y,Z):.2f}")
print(f"Permutation: \n\tstatistic value - {test_perm_results[0]:.1f}, \n\tp-value \t- {test_perm_results[1]:.3f}")
print(f"Aymptotic: \n\tstatistic value - {test_asymptotic_results[0]:.1f}, \n\tp-value \t- {test_asymptotic_results[1]:.3f}")

Conditional Mutual Information: 0.05
Permutation: 
	statistic value - 93.5, 
	p-value 	- 0.970
Aymptotic: 
	statistic value - 93.5, 
	p-value 	- 1.000


conditionaly dependent

In [123]:
X = np.round(np.random.normal(0, 1, N))
Z = np.random.choice([1,2,3,4], N)
Y = X * (Z > 2)

test_perm_results = conditional_permutations_of_X(X, Y, Z, B=100)
test_asymptotic_results = asymptotic_conditional_independence_test(X, Y, Z)

print(f"Conditional Mutual Information: {CMI(X,Y,Z):.2f}")
print(f"Permutation: \n\tstatistic value - {test_perm_results[0]:.1f}, \n\tp-value \t- {test_perm_results[1]:.3f}")
print(f"Aymptotic: \n\tstatistic value - {test_asymptotic_results[0]:.1f}, \n\tp-value \t- {test_asymptotic_results[1]:.3f}")

Conditional Mutual Information: 0.68
Permutation: 
	statistic value - 1364.9, 
	p-value 	- 0.010
Aymptotic: 
	statistic value - 1364.9, 
	p-value 	- 0.000


## Task 2

In [191]:
def sample_from_model1(N):
    Z_tilde = np.random.normal(0, 1, N)
    Z = np.where(Z_tilde < 0, -1, 1)
    X_tilde = np.array([np.random.normal(z/2, 1) for z in Z])
    X = np.where(X_tilde < 0, -1, 1)
    Y_tilde = np.array([np.random.normal(z/2, 1) for z in Z])
    Y = np.where(Y_tilde < 0, -1, 1)
    return X, Y, Z

def sample_from_model2(N):
    X_tilde = np.random.normal(0, 1, N)
    X = np.where(X_tilde < 0, -1, 1)
    Z_tilde = np.array([np.random.normal(x/2, 1) for x in X])
    Z = np.where(Z_tilde < 0, -1, 1)
    Y_tilde = np.array([np.random.normal(z/2, 1) for z in Z])
    Y = np.where(Y_tilde < 0, -1, 1)
    return X, Y, Z

def sample_from_model3(N):
    X_tilde = np.random.normal(0, 1, N)
    X = np.where(X_tilde < 0, -1, 1)
    Y_tilde = np.random.normal(0, 1, N)
    Y = np.where(Y_tilde < 0, -1, 1)
    Z_tilde = np.array([np.random.normal((x+y)/2, 1) for x, y in zip(X, Y)])
    Z = np.where(Z_tilde < 0, -1, 1)
    return X, Y, Z

### a)

Answer:
1. X and Y are not independent, but independent conditionally on Z,
2. X and Y are not independent, but independent conditionally on Z,
3. X and Y are independent, but dependent conditionally on Z,

### b)

In [206]:
N = 10000

data_model_1 = sample_from_model1(N)
data_model_2 = sample_from_model2(N)
data_model_3 = sample_from_model3(N)

fsinierivbsoervbaoiweucv = {
    "Model 1": data_model_1,
    "Model 2": data_model_2,
    "Model 3": data_model_3
}

results = []

for key, data in fsinierivbsoervbaoiweucv.items():
    results.append({"Model Name 1": key,
                    "MI_XY": mutual_info_score(*data[:2]),
                    "CMI_XY|Z": CMI(*data)})

pd.DataFrame(results)

,Model Name 1,MI_XY,CMI_XY|Z
0,Model 1,0.012270,0.000030
1,Model 2,0.011172,0.000002
2,Model 3,0.000009,0.009415


### c)

In [209]:
print("------------------------\n---Independence tests---\n------------------------")
print("Asymptotic test - MI")
print(f"Model 1: {indep_test_asymptotic(*data_model_1[:2], 'mi')}")
print(f"Model 2: {indep_test_asymptotic(*data_model_2[:2], 'mi')}")
print(f"Model 3: {indep_test_asymptotic(*data_model_3[:2], 'mi')}")
print("\nAsymptotic test - Pearson")
print(f"Model 1: {indep_test_asymptotic(*data_model_1[:2], 'chi2')}")
print(f"Model 2: {indep_test_asymptotic(*data_model_2[:2], 'chi2')}")
print(f"Model 3: {indep_test_asymptotic(*data_model_3[:2], 'chi2')}")
print("\nPermutation tests - MI")
print(f"Model 1: {indep_test_permutation(*data_model_1[:2], 100)}")
print(f"Model 2: {indep_test_permutation(*data_model_2[:2], 100)}")
print(f"Model 3: {indep_test_permutation(*data_model_3[:2], 100)}")

print("------------------------\n---Conditional Independence tests---\n------------------------")
print("Asymptotic test - CMI")
print(f"Model 1: {asymptotic_conditional_independence_test(*data_model_1)}")
print(f"Model 2: {asymptotic_conditional_independence_test(*data_model_2)}")
print(f"Model 3: {asymptotic_conditional_independence_test(*data_model_3)}")
print("\nPermutation tests - CMI")
print(f"Model 1: {conditional_permutations_of_X(*data_model_1, 100)}")
print(f"Model 2: {conditional_permutations_of_X(*data_model_2, 100)}")
print(f"Model 3: {conditional_permutations_of_X(*data_model_3, 100)}")

------------------------
---Independence tests---
------------------------
Asymptotic test - MI
Model 1: (245.40966072216008, 0.0)
Model 2: (223.44187719507192, 0.0)
Model 3: (0.18954073622623469, 0.6632991338938383)

Asymptotic test - Pearson
Model 1: (243.78260924612175, 0.0)
Model 2: (222.0120899399622, 0.0)
Model 3: (0.1725226731070022, 0.6778794912181693)

Permutation tests - MI
Model 1: (245.40966072216008, 0.009900990099009901)
Model 2: (223.44187719507192, 0.009900990099009901)
Model 3: (0.18954073622623469, 0.7326732673267327)
------------------------
---Conditional Independence tests---
------------------------
Asymptotic test - CMI
Model 1: (0.6038747726433833, 0.7393843590145955)
Model 2: (0.047500670249780996, 0.9765294844374284)
Model 3: (188.29695298548756, 0.0)

Permutation tests - CMI
Model 1: (0.6038747726433833, 0.7128712871287128)
Model 2: (0.047500670249780996, 0.9801980198019802)
Model 3: (188.29695298548756, 0.009900990099009901)


Based on p-values:
- In models 1 and 2: X and Y are not independent, but are independent conditionally on Z,
- In model 3: X and Y are independent, but dependent conditionally on Z.

Different tests point to the same conclusions.

### d) *

In [217]:
def sample_from_model1_d(N):
    Z_tilde = np.random.normal(0, 1, N)
    X_tilde = np.array([np.random.normal(z_tilde/2, 1) for z_tilde in Z_tilde])
    Y_tilde = np.array([np.random.normal(z_tilde/2, 1) for z_tilde in Z_tilde])
    Z = np.where(Z_tilde < 0, -1, 1)
    X = np.where(X_tilde < 0, -1, 1)
    Y = np.where(Y_tilde < 0, -1, 1)
    return X, Y, Z

def sample_from_model2_d(N):
    X_tilde = np.random.normal(0, 1, N)
    Z_tilde = np.array([np.random.normal(x_tilde/2, 1) for x_tilde in X_tilde])
    Y_tilde = np.array([np.random.normal(z_tilde/2, 1) for z_tilde in Z_tilde])
    X = np.where(X_tilde < 0, -1, 1)
    Z = np.where(Z_tilde < 0, -1, 1)
    Y = np.where(Y_tilde < 0, -1, 1)
    return X, Y, Z

def sample_from_model3_d(N):
    X_tilde = np.random.normal(0, 1, N)
    Y_tilde = np.random.normal(0, 1, N)
    Z_tilde = np.array([np.random.normal((x_tilde+y_tilde)/2, 1) for x_tilde, y_tilde in zip(X_tilde, Y_tilde)])
    X = np.where(X_tilde < 0, -1, 1)
    Y = np.where(Y_tilde < 0, -1, 1)
    Z = np.where(Z_tilde < 0, -1, 1)
    return X, Y, Z

In [236]:
N = 10000

data_model_1_d = sample_from_model1_d(N)
data_model_2_d = sample_from_model2_d(N)
data_model_3_d = sample_from_model3_d(N)

fsinierivbsoervbaoiweucv_d = {
    "Model 1": data_model_1_d,
    "Model 2": data_model_2_d,
    "Model 3": data_model_3_d
}

results_d = []

for key, data in fsinierivbsoervbaoiweucv_d.items():
    results_d.append({"Model Name 1": key,
                    "MI_XY": mutual_info_score(*data[:2]),
                    "CMI_XY|Z": CMI(*data)})

pd.DataFrame(results_d)

,Model Name 1,MI_XY,CMI_XY|Z
0,Model 1,0.006082,0.000554
1,Model 2,0.008973,0.000740
2,Model 3,0.000027,0.003668


In [238]:
print("------------------------\n---Independence tests---\n------------------------")
print("Asymptotic test - MI")
print(f"Model 1: {indep_test_asymptotic(*data_model_1_d[:2], 'mi')}")
print(f"Model 2: {indep_test_asymptotic(*data_model_2_d[:2], 'mi')}")
print(f"Model 3: {indep_test_asymptotic(*data_model_3_d[:2], 'mi')}")
print("\nAsymptotic test - Pearson")
print(f"Model 1: {indep_test_asymptotic(*data_model_1_d[:2], 'chi2')}")
print(f"Model 2: {indep_test_asymptotic(*data_model_2_d[:2], 'chi2')}")
print(f"Model 3: {indep_test_asymptotic(*data_model_3_d[:2], 'chi2')}")
print("\nPermutation tests - MI")
print(f"Model 1: {indep_test_permutation(*data_model_1_d[:2], 100)}")
print(f"Model 2: {indep_test_permutation(*data_model_2_d[:2], 100)}")
print(f"Model 3: {indep_test_permutation(*data_model_3_d[:2], 100)}")

print("------------------------\n---Conditional Independence tests---\n------------------------")
print("Asymptotic test - CMI")
print(f"Model 1: {asymptotic_conditional_independence_test(*data_model_1_d)}")
print(f"Model 2: {asymptotic_conditional_independence_test(*data_model_2_d)}")
print(f"Model 3: {asymptotic_conditional_independence_test(*data_model_3_d)}")
print("\nPermutation tests - CMI")
print(f"Model 1: {conditional_permutations_of_X(*data_model_1_d, 100)}")
print(f"Model 2: {conditional_permutations_of_X(*data_model_2_d, 100)}")
print(f"Model 3: {conditional_permutations_of_X(*data_model_3_d, 100)}")

------------------------
---Independence tests---
------------------------
Asymptotic test - MI
Model 1: (121.64280410902117, 0.0)
Model 2: (179.46374380909847, 0.0)
Model 3: (0.5454932336257379, 0.4601650260715868)

Asymptotic test - Pearson
Model 1: (120.95598077394999, 0.0)
Model 2: (178.3927267199013, 0.0)
Model 3: (0.5163450171872781, 0.47240497166465745)

Permutation tests - MI
Model 1: (121.64280410902117, 0.009900990099009901)
Model 2: (179.46374380909847, 0.009900990099009901)
Model 3: (0.5454932336257379, 0.49504950495049505)
------------------------
---Conditional Independence tests---
------------------------
Asymptotic test - CMI
Model 1: (11.084930588375567, 0.0039168587167914826)
Model 2: (14.794943740558534, 0.0006128000424416857)
Model 3: (73.35402352204817, 1.1102230246251565e-16)

Permutation tests - CMI
Model 1: (11.084930588375567, 0.009900990099009901)
Model 2: (14.794943740558534, 0.009900990099009901)
Model 3: (73.35402352204817, 0.009900990099009901)


Changed:
- in all models, X and Y are now conditionally dependent,
- p-value for independence of X and Y in model 3 is slightly smaller than earlier.